기존 임베딩 vs BERT 임베딩  
  
- 기존 임베딩은 문맥에 상관없이 같은 단어면 항상 같은 임베딩   
  
- BERT 임베딩은 문맥에 따라 달라짐 (동음이의어뿐만 아니라, 문맥 달라지면 같은 의미의 단어도 임베딩 달라짐)  

1. Loading Pre-Trained BERT

설치 : pip install transformers

In [ ]:
# ! pip install transformers

In [2]:
# import pytorch, the pretrained BERT model, and a BERT tokenizer.
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')


Downloading: 100%|██████████| 232k/232k [00:00<00:00, 383kB/s]


# 2. Input Formatting  
1. A special token, [SEP], to mark the end of a sentence, or the separation between two sentences
2. A special token, [CLS], at the beginning of our text. This token is used for classification tasks, but BERT expects it no matter what your application is.
3. Tokens that conform with the fixed vocabulary used in BERT
4. The Token IDs for the tokens, from BERT’s tokenizer   
5.Mask IDs to indicate which elements in the sequence are tokens and which are padding   
elements  
6. Segment IDs used to distinguish different sentences  
7. Positional Embeddings used to show token position within the sequence  


In [9]:
import pandas as pd
import ast

In [5]:
news = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_train_all.csv')
news.head()

,url,nids,Title,Date,Content,Image
0,https://assets.msn.com/labs/mind/AABmf2I.html,AABmf2I,Walmart Slashes Prices on Last-Generation iPads,2019-10-28T17:38:04Z,"['This year, Walmartâ\x80\x99s not waiting unt...",https://img-s-msn-com.akamaized.net/tenant/amp...
1,https://assets.msn.com/labs/mind/AAISxPN.html,AAISxPN,Dispose of unwanted prescription drugs on Satu...,2019-10-25T17:31:42Z,"['CINCINNATI (FOX19) - If you have expired, un...",https://img-s-msn-com.akamaized.net/tenant/amp...
2,https://assets.msn.com/labs/mind/AAEKnO1.html,AAEKnO1,Check Houston traffic map for current road con...,2019-11-26T12:55:03Z,"[""Get real-time traffic data from Houston's ro...",https://img-s-msn-com.akamaized.net/tenant/amp...
3,https://assets.msn.com/labs/mind/AADlomf.html,AADlomf,"How to record your screen on Windows, macOS, i...",2019-11-06T16:06:25Z,"[""There are lots of reasons to record your scr...",https://img-s-msn-com.akamaized.net/tenant/amp...
4,https://assets.msn.com/labs/mind/AAGBR44.html,AAGBR44,This Guy Altered His Diet and Training to Drop...,2019-08-31T11:37:11Z,['Brandon Reid always felt like â\x80\x9cthe f...,https://img-s-msn-com.akamaized.net/tenant/amp...


In [74]:
def content(x):
    try:
        return ' '.join(ast.literal_eval(x)) 
    except:
        return None

In [75]:
news_contents = news.loc[:,'Content'].apply(lambda x: content(x))
news_contents[0] 

'This year, Walmartâ\x80\x99s not waiting until Black Friday to offer steep deals on tech. Right now, you can save big on last-generation iPads since Apple just released new models for 2019. Whether youâ\x80\x99re looking to upgrade your device or get a holiday gift for someone else, there are a variety of standard iPads, iPad Airs, and iPad Pros to consider. The main difference between sixth- and seventh-generation iPads is the new iPad OS software, but not too much has changed in terms of overall design and power. The 2018 iPads all feature the powerful A10 Fusion chip, iOS 11, 1.2MP front and 8MP rear cameras, Touch ID, and 10-hour battery life.Sixth-generation iPads used to cost up to and over $500, but now you can score one for as low as $300. The 9.7-inch Wi-Fi model with 128GB is now at $300, while the version with cellular capabilities is just $430. The iPad mini with a screen size of just 7.9 inches still costs over $500, but you get top features like the A12 chip, Retina disp

In [88]:
news.loc[50243:,'Content']

50243     The royals are free to shop wherever they choo...
50244     When you first start dieting and exercising, t...
50245      ZOLOTE, Ukraine â Lt. Ivan Molchanets peeke...
50246     Walking into the Bradley Center, I could feel ...
50247      [Video by Health.com] As you get older, littl...
                                ...                        
101516    MALIBU, CA â November 8, 2019, marked the on...
101517    Eyewitness News is learning what caused the de...
101518    Sometimes, what happens on the sidelines can b...
101519    The Seattle Sounders just won MLS Cup for the ...
101520    \n As fall temperatures cool down in much of t...
Name: Content, Length: 51278, dtype: object

In [89]:
news.loc[50243:,'Content'].iloc[0]

'The royals are free to shop wherever they choose, but they tend to keep it in the familyâ\x80\x94the royal warrant family, that is. The precious few brands with royal warrants are permitted display the Royal Arms in their marketing materials. The royals usually refuse to promote commercial goods, but the warrants are a rare and prominent exception. They\'re an obvious boon to any branding efforts, but getting your hands on one is no easy task.  First, a company must provide goods or services to the royal family (or more specifically, to Queen Elizabeth, Prince Philip, and Prince Charles, the three royals able to award warrants) for five years. Then, as Nigel Fulton, owner of royal warrantee Fulton Umbrellas, told Town & Country earlier this year, "you have to wait until youâ\x80\x99re invited to apply, so itâ\x80\x99s not automatic at all." To apply, companies need to submit a laundry list of disclosures, which ensure that the business meets various standards. "The finances, your supp

In [50]:
def special_token(content):
    new_lst = []
    for x in content.split('.')[:-1]: 
        new_lst.append(x + " [SEP]")

    return "[CLS] " + ' '.join(new_lst)

In [51]:
marked_text = news_contents.apply(lambda x: special_token(x))
marked_text[0]

'[CLS] This year, Walmartâ\x80\x99s not waiting until Black Friday to offer steep deals on tech [SEP]  Right now, you can save big on last-generation iPads since Apple just released new models for 2019 [SEP]  Whether youâ\x80\x99re looking to upgrade your device or get a holiday gift for someone else, there are a variety of standard iPads, iPad Airs, and iPad Pros to consider [SEP]  The main difference between sixth- and seventh-generation iPads is the new iPad OS software, but not too much has changed in terms of overall design and power [SEP]  The 2018 iPads all feature the powerful A10 Fusion chip, iOS 11, 1 [SEP] 2MP front and 8MP rear cameras, Touch ID, and 10-hour battery life [SEP] Sixth-generation iPads used to cost up to and over $500, but now you can score one for as low as $300 [SEP]  The 9 [SEP] 7-inch Wi-Fi model with 128GB is now at $300, while the version with cellular capabilities is just $430 [SEP]  The iPad mini with a screen size of just 7 [SEP] 9 inches still costs 

In [42]:
# Tokenize our sentence with the BERT tokenizer.
tokenized_texts = []
for text in marked_text:
    tokenized_texts.append(tokenizer.tokenize(text))

# Print out the tokens.
print(tokenized_texts[0])

KeyboardInterrupt: 

In [ ]:
list(tokenizer.vocab.keys())[5000:5020]

In [ ]:
# Map the token strings to their vocabulary indeces.
indexed_tokens = []
for tokenized_text in tokenized_texts:
    indexed_token = tokenizer.convert_tokens_to_ids(tokenized_text)
    indexed_tokens.append(indexed_token)

print(indexed_tokens[0])    

In [ ]:
# Display the words with their indeces.
for tup in zip(tokenized_texts[0], indexed_tokens[0]):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

# 3. Segment ID  
BERT is trained on and expects sentence pairs, using 1s and 0s to distinguish between the two sentences.   
  
If you want to process two sentences, assign each word in the first sentence plus the ‘[SEP]’ token a 0, and all tokens of the second sentence a 1.

In [ ]:
def segid(text):
    [0] * len(text[0]) 

In [ ]:
for tokenized_text in tokenized_texts:
    [0] * len(tokenized_text[0])


In [ ]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

In [52]:
! pip install -U sentence-transformers

     |████████████████████████████████| 65 kB 764 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.2-py3-none-any.whl size=93964 sha256=796b84a9cbb6a16f0bc6db3deb34b613b195c5a049a7c4459834c37aebb1c8f6
  Stored in directory: /Users/baeyuna/Library/Caches/pip/wheels/8d/0f/5e/f156d6e1e5d0e9871a99c2c6ef3e07ce3e26511f71aa998bb6
Successfully built sentence-transformers


In [53]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [02:16<00:00, 2.97MB/s]


In [54]:
news_contents[0]

'This year, Walmartâ\x80\x99s not waiting until Black Friday to offer steep deals on tech. Right now, you can save big on last-generation iPads since Apple just released new models for 2019. Whether youâ\x80\x99re looking to upgrade your device or get a holiday gift for someone else, there are a variety of standard iPads, iPad Airs, and iPad Pros to consider. The main difference between sixth- and seventh-generation iPads is the new iPad OS software, but not too much has changed in terms of overall design and power. The 2018 iPads all feature the powerful A10 Fusion chip, iOS 11, 1.2MP front and 8MP rear cameras, Touch ID, and 10-hour battery life.Sixth-generation iPads used to cost up to and over $500, but now you can score one for as low as $300. The 9.7-inch Wi-Fi model with 128GB is now at $300, while the version with cellular capabilities is just $430. The iPad mini with a screen size of just 7.9 inches still costs over $500, but you get top features like the A12 chip, Retina disp

In [62]:
news_contents[2]

'This year, Walmartâ\x80\x99s not waiting until Black Friday to offer steep deals on tech. Right now, you can save big on last-generation iPads since Apple just released new models for 2019. Whether youâ\x80\x99re looking to upgrade your device or get a holiday gift for someone else, there are a variety of standard iPads, iPad Airs, and iPad Pros to consider. The main difference between sixth- and seventh-generation iPads is the new iPad OS software, but not too much has changed in terms of overall design and power. The 2018 iPads all feature the powerful A10 Fusion chip, iOS 11, 1.2MP front and 8MP rear cameras, Touch ID, and 10-hour battery life.Sixth-generation iPads used to cost up to and over $500, but now you can score one for as low as $300. The 9.7-inch Wi-Fi model with 128GB is now at $300, while the version with cellular capabilities is just $430. The iPad mini with a screen size of just 7.9 inches still costs over $500, but you get top features like the A12 chip, Retina disp

In [55]:
sentence_embeddings = model.encode(news_contents[0])

In [57]:
for sentence, embedding in zip(news_contents[0], sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: T
Embedding: [-4.25543487e-01  7.60001302e-01  9.18531656e-01 -2.95201480e-01
  5.87620556e-01 -8.64016652e-01  5.59141576e-01  3.80701602e-01
  3.97778451e-01 -6.16761982e-01  3.48098993e-01  5.42076945e-01
  3.77421051e-01 -1.62790880e-01 -7.97211528e-01  4.97375309e-01
  3.04683357e-01  2.90414721e-01 -1.31681543e-02 -1.72260597e-01
 -2.21253678e-01 -6.86304927e-01  5.34278274e-01  2.51835823e-01
  1.01992214e+00  2.51798868e-01 -1.26365617e-01  1.37554824e-01
 -3.21952701e-01  5.16393840e-01 -5.18519759e-01  6.90127730e-01
 -5.36325455e-01 -6.29978180e-01  1.13740213e-01  1.08845699e+00
 -2.36613840e-01  3.33904296e-01  1.79698452e-01  1.80658951e-01
  4.84353453e-02 -9.61049914e-01 -2.02181622e-01 -3.47175151e-01
 -1.16442001e+00 -2.41194367e-02 -1.49435312e-01  3.38092744e-01
  5.86021602e-01 -6.75692320e-01  9.13677573e-01  5.71824551e-01
  4.18858081e-01 -4.24213916e-01 -5.86561680e-01  8.15579474e-01
  3.44417900e-01 -8.98973227e-01 -5.23578942e-01 -6.00768387e-01
 -

In [59]:
from sklearn.cluster import KMeans

num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sentence_embeddings)
cluster_assignment = clustering_model.labels_

ValueError: n_samples=1 should be >= n_clusters=5